# Phase 3 – Modélisation & Enrichissement ML

Objectif : enrichir le dataset nettoyé (Phase 2) avec
- **Topics NMF** pour regrouper les offres par thématiques
- **Classe salaire** (haut/bas) pour l’analyse et le dashboard
- **Jeu de données enrichi** prêt pour Phase 4

## Étape 1 – Importer les bibliothèques

- pandas / numpy
- scikit-learn (TF-IDF, NMF, LogisticRegression, métriques)
- utils (Path)
- tqdm pour le confort d’affichage
- Warnings filtrés pour un notebook plus lisible

## Étape 2 – Charger le dataset nettoyé (Phase 2)

On part du fichier produit en Phase 2 : `data/processed/hellowork_cleaned.csv`.

In [22]:
# --- Imports ---
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

print("✅ Imports OK")

✅ Imports OK


## Étape 3 – Harmoniser les colonnes utiles

On crée/normalise les colonnes attendues pour le ML :
- `description_clean` (texte prêt pour TF-IDF)
- `salary_monthly` (valeur numérique mensuelle)
- colonnes catégorielles encodées si présentes (`sector_enc`, `location_enc`, etc.)

In [23]:
# --- Charger le dataset nettoyé ---
clean_path = Path("data/processed/hellowork_cleaned.csv")
assert clean_path.exists(), "Le fichier nettoyé de la Phase 2 est introuvable."

df = pd.read_csv(clean_path, encoding="utf-8")
print(f"📊 Shape: {df.shape}")
print(f"📋 Colonnes: {df.columns.tolist()}")

df.head(3)

📊 Shape: (1239, 15)
📋 Colonnes: ['Sector', 'Job_Title', 'Company', 'Location', 'Contract', 'Salary', 'Description', 'URL', 'Salary_Monthly', 'Description_Clean', 'Top_Keywords', 'Sector_enc', 'Location_enc', 'Contract_enc', 'Company_enc']


,Sector,Job_Title,Company,Location,Contract,Salary,Description,URL,Salary_Monthly,Description_Clean,Top_Keywords,Sector_enc,Location_enc,Contract_enc,Company_enc
0,Agriculture • Pêche,Alternance - Chargé·e de Formation H/F,Remy Cointreau,Paris - 75,Alternance,"486,49 - 1 801,80 € / mois",Nous recherchons un·e candidat·e : Alternance...,https://www.hellowork.com/fr-fr/emplois/642118...,1144.145,recherchons un·e candidat·e alternance chargé·...,"formation,formations,groupe,aider,plan,créatio...",0,0,0,0
1,BTP,Electricien H/F,Samsic Emploi,Rennes - 35,Intérim,12 - 15 € / heure,Nous recherchons activement un/une electricien...,https://www.hellowork.com/fr-fr/emplois/729658...,2160.000,recherchons activement unune electriciennne ca...,"travail,samsic,sengage,lun,passion,lexpérience...",1,1,1,1
2,BTP,Ouvrier Polyvalent en Menuiserie H/F,Groupe Actual,Auterive - 31,Intérim,"Estimation → 12,36 - 13,50 € / heure",Nous recherchons un(e) menuisier(e) expériment...,https://www.hellowork.com/fr-fr/emplois/732798...,2068.800,recherchons menuisiere expérimentée rejoindre ...,"recherchons,dexpérience,connaissance,candidats...",1,2,1,2


## Étape 4 – Créer la cible salaire (High vs Low)

On utilise la médiane du salaire mensuel pour définir `high_salary` (1 = au-dessus de la médiane).

In [24]:
# --- Harmoniser les noms de colonnes ---
# Standardize column names to match expected format

col_map = {
    "Description_Clean": "description_clean",
    "Salary_Monthly": "salary_monthly",
    "Sector_enc": "sector_enc",
    "Location_enc": "location_enc",
    "Contract_enc": "contract_enc",
    "Company_enc": "company_enc",
}

df = df.rename(columns={k: v for k, v in col_map.items() if k in df.columns})

# Guarantee minimal required columns
if "description_clean" not in df.columns:
    if "Description" in df.columns:
        df["description_clean"] = df["Description"].fillna("").astype(str)
    else:
        df["description_clean"] = ""

if "salary_monthly" not in df.columns:
    if "Salary" in df.columns:
        df["salary_monthly"] = pd.to_numeric(df["Salary"], errors='coerce')
    else:
        df["salary_monthly"] = np.nan

print("✅ Colonnes harmonisées")
print(f"📊 Colonnes disponibles: {df.columns.tolist()}")
print(f"\n📋 Aperçu des colonnes clés:")
print(df[[c for c in ["description_clean", "salary_monthly"] if c in df.columns]].head(3))

✅ Colonnes harmonisées
📊 Colonnes disponibles: ['Sector', 'Job_Title', 'Company', 'Location', 'Contract', 'Salary', 'Description', 'URL', 'salary_monthly', 'description_clean', 'Top_Keywords', 'sector_enc', 'location_enc', 'contract_enc', 'company_enc']

📋 Aperçu des colonnes clés:
                                   description_clean  salary_monthly
0  recherchons un·e candidat·e alternance chargé·...        1144.145
1  recherchons activement unune electriciennne ca...        2160.000
2  recherchons menuisiere expérimentée rejoindre ...        2068.800


## Étape 5 – Clustering thématique avec NMF

Pipeline :
1) TF-IDF sur `description_clean`
2) NMF (5 topics) pour extraire les thématiques
3) Attribution du cluster dominant dans `job_cluster`
4) Top mots par topic pour interprétation

In [25]:
# --- Cible binaire salaire (high vs low) ---
median_salary = df["salary_monthly"].median(skipna=True)
df["high_salary"] = (df["salary_monthly"] > median_salary).astype(int)

print(f"💰 Médiane salaire mensuel: €{median_salary:.2f}")
print(f"\n📊 Distribution de la cible:")
print(df["high_salary"].value_counts())
print(f"\n✓ Salaires valides: {df['salary_monthly'].notna().sum()}/{len(df)}")

💰 Médiane salaire mensuel: €2116.90

📊 Distribution de la cible:
high_salary
0    811
1    428
Name: count, dtype: int64

✓ Salaires valides: 1073/1239


## Étape 6 – Classification salaire (Logistic Regression)

- Données d’entrée : TF-IDF sur `description_clean`
- Cible : `high_salary`
- Métriques : rapport de classification + AUC ROC

In [26]:
# --- TF-IDF + NMF Clustering ---
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

text_col = "description_clean"
texts = df[text_col].fillna("").astype(str)

print("🔄 Création de la matrice TF-IDF...")
vect = TfidfVectorizer(max_features=1000, min_df=2, max_df=0.9, ngram_range=(1, 2))
X_text = vect.fit_transform(texts)
print(f"✓ Matrice TF-IDF: {X_text.shape}")

print("\n🔄 Application de NMF (7 topics)...")
nmf = NMF(n_components=7, random_state=42, max_iter=500, init='nndsvda')
W = nmf.fit_transform(X_text)

# Assign each job to dominant topic
df["job_cluster"] = W.argmax(axis=1)
print("✓ Clustering NMF terminé")
print(f"\n📊 Répartition des clusters:")
print(df["job_cluster"].value_counts().sort_index())

# Interpretation of topics
def print_topics(nmf_model, feature_names, n_top_words=10):
    print("\n🔑 Interprétation des topics (top 10 mots):")
    for i, comp in enumerate(nmf_model.components_):
        terms = [feature_names[j] for j in comp.argsort()[-n_top_words:]]
        print(f"  Topic {i}: {', '.join(terms)}")

print_topics(nmf, vect.get_feature_names_out())

🔄 Création de la matrice TF-IDF...
✓ Matrice TF-IDF: (1239, 1000)

🔄 Application de NMF (7 topics)...
✓ Clustering NMF terminé

📊 Répartition des clusters:
job_cluster
0    209
1    149
2    210
3    103
4    404
5     70
6     94
Name: count, dtype: int64

🔑 Interprétation des topics (top 10 mots):
  Topic 0: journée, heure, pourrez, quelques, brut heure, collège, élèves niveau, cours, niveau, élèves
  Topic 1: rejoins, déplacement, engagés, frais déplacement, laventure, rejoins laventure, laventure esallia, produits engagés, produits, esallia
  Topic 2: kids, jeux, intelligente, domicile, mission, activités, garde denfants, garde, denfants, enfants
  Topic 3: chez, première, première année, année, diplômes, salaire, 1418, missions, ouihelp, aide
  Topic 4: suivi, missions, sein, recrutement, formation, clients, participer, poste, gestion, développement
  Topic 5: opérationnelle, indicateurs, gestion opérationnelle, gestion, vente relation, client gestion, tenue, relation client, clie

### b) Entraînement du modèle

In [27]:
# --- Classification salaire (Logistic Regression) ---
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# Filter to rows with valid labels
df_train = df[df["high_salary"].notna() & df[text_col].notna()].copy()
print(f"📊 Données d'entraînement: {len(df_train)} échantillons")

if len(df_train) > 10:
    # Use the same TF-IDF matrix (X_text) but filter to training rows
    # FIX: Convert boolean mask to numpy array for sparse matrix indexing
    mask = (df["high_salary"].notna() & df[text_col].notna()).values
    X = X_text[mask]
    y = df_train["high_salary"].values
    
    print(f"\n🔄 Séparation train/test (80/20)...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"✓ Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")
    
    print("\n🔄 Entraînement du modèle (Logistic Regression)...")
    clf = LogisticRegression(max_iter=1000, C=10, solver='lbfgs', class_weight='balanced')
    clf.fit(X_train, y_train)
    
    print("✓ Modèle entraîné")
    
    # Evaluate
    preds = clf.predict(X_test)
    probs = clf.predict_proba(X_test)[:, 1]
    
    print(f"\n📊 Rapport de classification:")
    print(classification_report(y_test, preds, digits=3))
    
    auc = roc_auc_score(y_test, probs)
    print(f"\n🎯 AUC-ROC: {auc:.4f}")
else:
    print("⚠️  Données insuffisantes pour la classification")
    clf = None

📊 Données d'entraînement: 1239 échantillons

🔄 Séparation train/test (80/20)...
✓ Train: 991 | Test: 248

🔄 Entraînement du modèle (Logistic Regression)...
✓ Modèle entraîné

📊 Rapport de classification:
              precision    recall  f1-score   support

           0      0.925     0.994     0.958       162
           1      0.986     0.849     0.912        86

    accuracy                          0.944       248
   macro avg      0.956     0.921     0.935       248
weighted avg      0.947     0.944     0.942       248


🎯 AUC-ROC: 0.9710


## Étape 7 – Attacher les prédictions

On génère les prédictions sur l’ensemble du dataset et on ajoute:
- `pred_high_salary` (classe prédite)
- `pred_proba_high_salary` (probabilité associée)

In [28]:
# --- Prédictions sur l'ensemble complet ---
if clf is not None:
    print("🔄 Génération des prédictions sur le dataset complet...")
    df["predicted_high_salary"] = clf.predict(X_text)
    df["pred_proba_high_salary"] = clf.predict_proba(X_text)[:, 1]
    print("✓ Prédictions ajoutées")
    
    print("\n📊 Aperçu des prédictions:")
    display_cols = ["salary_monthly", "high_salary", "predicted_high_salary", "pred_proba_high_salary"]
    print(df[display_cols].head(10))
    
    # Calculate prediction accuracy on labeled data
    labeled_df = df[df["high_salary"].notna()]
    accuracy = (labeled_df["high_salary"] == labeled_df["predicted_high_salary"]).mean()
    print(f"\n✓ Précision globale sur données étiquetées: {accuracy:.2%}")
else:
    df["predicted_high_salary"] = -1
    df["pred_proba_high_salary"] = 0.0
    print("⚠️  Pas de prédictions (modèle non entraîné)")

🔄 Génération des prédictions sur le dataset complet...
✓ Prédictions ajoutées

📊 Aperçu des prédictions:
   salary_monthly  high_salary  predicted_high_salary  pred_proba_high_salary
0        1144.145            0                      0                0.018902
1        2160.000            1                      0                0.212084
2        2068.800            0                      0                0.039824
3             NaN            0                      0                0.043275
4        2080.000            0                      0                0.012916
5        1144.145            0                      0                0.025276
6        2240.000            1                      1                0.708790
7        1900.800            0                      0                0.084087
8        2200.000            1                      0                0.428402
9             NaN            0                      0                0.089906

✓ Précision globale sur données étiq

## Étape 8 – Sauvegarder le dataset enrichi

On enregistre le résultat pour le dashboard (Phase 4) :
- Fichier: `data/enriched/hellowork_ml_enriched.csv`
- Contient: texte nettoyé, clusters, labels salaire, prédictions

In [29]:
# --- Sauvegarde du dataset enrichi ---
from pathlib import Path

output_path = Path("data/enriched/hellowork_ml_enriched.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False, encoding="utf-8")

print("=" * 70)
print("✅ PHASE 3 COMPLETE")
print("=" * 70)
print(f"\n📁 Fichier enrichi sauvegardé: {output_path}")
print(f"📊 Lignes: {len(df)}")
print(f"📋 Colonnes: {len(df.columns)}")
print(f"\n🆕 Nouvelles colonnes ajoutées:")
print(f"   • job_cluster (0-6): Topics NMF")
print(f"   • high_salary (0/1): Label salaire (médiane)")
print(f"   • predicted_high_salary (0/1): Prédiction du modèle")
print(f"   • pred_proba_high_salary: Probabilité prédite")
print("\n✓ Prêt pour la Phase 4 (Dashboard)")
print("=" * 70)

✅ PHASE 3 COMPLETE

📁 Fichier enrichi sauvegardé: data\enriched\hellowork_ml_enriched.csv
📊 Lignes: 1239
📋 Colonnes: 19

🆕 Nouvelles colonnes ajoutées:
   • job_cluster (0-6): Topics NMF
   • high_salary (0/1): Label salaire (médiane)
   • predicted_high_salary (0/1): Prédiction du modèle
   • pred_proba_high_salary: Probabilité prédite

✓ Prêt pour la Phase 4 (Dashboard)
